# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов переданы в виде csv-файла.
Данные представлены в виде таблищы столбцы которое содержат следующую информацию:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
Нам предстоит ответить на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?


План исследования:
1. [Шаг 1.Откроем файл с данными и изучим общую информацию](#start)
2. [Шаг 2. Предобработка данных](#preprocessing)
* [Обработка пропусков](#null)
* [Замена типа данных](#replacing_type)
* [Обработка дубликатов](#duplicates)
* [Лемматизация](#lemmatization)
* [Категоризация данных](#categorization)
3. [Шаг 3. Ответим на вопросы](#questions)
4. [Шаг 4. Общий вывод](#conclusion)


<a id="start"></a>
##  Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
#Импортируем библиотеки необходимые для исследования
import pandas as pd
data = pd.read_csv('data.csv')
from pymystem3 import Mystem 
m = Mystem() 
from collections import Counter 

display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Датасет содержит 12 столбцов и 21525 строк
1. Названия столбцов - ок
2. Типы данных в 1/10 столцов тип данных не соответствует содержанию заявленному в названии столбца:
- Общий трудовой стаж (days_employed)- тип данных вещественные числа(float) нужно заменить на целые числа(int)
3. В 2 из 12 столбцов содержатся нулевые значения:
- days_employed и total_income содержат нулевые значения и потребуется обработка пропусков. Оба параметра количественные.

4. Столбец 'family_status' можно удалить, т.к. он дублирует информацию в 'family_status_id' и хранит информцию в сложном виде.

5. Столбец 'days_employed'(трудовой стаж в днях) содержит экстремально большие значения. Можно предположить, что часть данных указана в часах а не в днях
6. В столбце 'education' значения записаны в разном регистре, стоит изменить написание всех значений на строчные буквы.

<a id="preprocessing"></a>
## Шаг 2. Предобработка данных 

<a id="null"></a>
### Обработка пропусков и аномальных значений

In [2]:
# Найдем нулевые значения количественных переменных
print(data[data['days_employed'] == 0]['days_employed'].count())
print(data[data['total_income'] == 0]['total_income'].count())
print(data[data['dob_years'] == 0]['dob_years'].count()) 


0
0
101


Количество строк с нулевым значением меньше одного процента. Их можно удалить без значительного внияния на результаты анализа

In [3]:
data = data[data['dob_years'] != 0]

В столбце мы обрнаружили аномальные значения "20", где верояно пропущена точка (2.0) и "-1" с ошибочным знаком.

In [4]:
#print(data['children'].unique()) # В столбце мы обрнаружили аномальные значения "20", где верояно пропущена точка (2.0) и "-1" с ошибочным знаком.
# Устарним некорректные значения в этом столбще
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2

#print(data['gender'].unique()) # В столбще обнаружено некорректное значение пола - "XNA"
#print(data.loc[data['gender'] == 'XNA', 'gender'].count()) # Встречается единожды. 
data = data.loc[data['gender'] != 'XNA'] #Стоку можно удалить без значительного внияния на результаты анализа

# В столбце 'days_employed' мы обраружили отрицательные и аномально большие значения.Аномально большие данные видимо указаны в часах и поступили из другой системы.
# Для приведения к единому стилю, разделим эти значения на 24 и прведем к положительным значениям. Средний трудовой стаж в РФ при выходе на пенсию составляет 35 лет(или 12 775 дней)
#Будем считать это верхней границей, т.е. допустим что все значения больше 12 775 указаны в часах и должны быть пересчитаны в  дни
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] > 12775, 'days_employed'] = data.loc[data['days_employed'] > 12775, 'days_employed'] / 24

# Найдем пропущеные значения 
#print(data.isna().sum())

# Допустим что трудовой стаж и ежемесячный доход напрямую зависит от возраста, поэтому заполним пропущеные значения медианным значением с группировакой по переменной возраст. 
for year in data['dob_years'].unique():
    median_days_employed = data.loc[data['dob_years'] == year, 'days_employed'].median()
    median_total_income = data.loc[data['dob_years'] == year, 'total_income'].median()
    data.loc[(data['days_employed'].isna()) & (data['dob_years'] == year), 'days_employed'] = median_days_employed
    data.loc[(data['total_income'].isna()) & (data['dob_years'] == year), 'total_income'] = median_total_income

#data.head(20)
#data.isna().sum() 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21423 non-null  int64  
 1   days_employed     21423 non-null  float64
 2   dob_years         21423 non-null  int64  
 3   education         21423 non-null  object 
 4   education_id      21423 non-null  int64  
 5   family_status     21423 non-null  object 
 6   family_status_id  21423 non-null  int64  
 7   gender            21423 non-null  object 
 8   income_type       21423 non-null  object 
 9   debt              21423 non-null  int64  
 10  total_income      21423 non-null  float64
 11  purpose           21423 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


**Вывод**

Мы обнаружили:
1. Нулевые значения в столбце 'dob_years' (возраст клиента). Значений было меньше одного процента, что значительно не повлияет на результаты. Принято решение удалить строки полностью. 
2. В столбце 'children' обрнаружили аномальные значения "20", где верояно пропущена точка (2.0) и "-1" с ошибочным знаком. Заменили.
3. В столбще 'gender' обнаружено некорректное значение пола - "XNA". 
4. В столбце 'days_employed' браружили отрицательные и аномально большие значения.Аномально большие данные видимо указаны в часах и поступили из другой системы. В качестве верхней границы стажа приняли средний трудовой стаж в РФ при выходе на пенсию. Данные выше этой границы пересчитали в дни. В отрицательных значениях заменили знак. 
5. В столбцах 'days_employed' и 'total_income' обнаружили пропущенные значения. Значений достаточно много (окололо 10%), возможно проблема с выгрузкой данных и стоило бы пообщаться с разработчиками. Удалить такое количество строк без ущерба результату мы не можем. Принято решение заполнить пропуски медианным значением. Принято допущение, что трудовой стаж и ежемесячный доход напрямую зависят от возраста, поэтому пропущеные значения заполнены медианным значением с группировакой по переменной возраст.

Пропуски обработаны.


<a id="replacing_type"></a> 
### Замена типа данных

In [5]:
# В столбцах 'days_employed' и 'total_income' значения имют вещественный тип данных. Изменим тип данных на целочисленный универсальным методом astype(). 
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21423 non-null  int64 
 1   days_employed     21423 non-null  int64 
 2   dob_years         21423 non-null  int64 
 3   education         21423 non-null  object
 4   education_id      21423 non-null  int64 
 5   family_status     21423 non-null  object
 6   family_status_id  21423 non-null  int64 
 7   gender            21423 non-null  object
 8   income_type       21423 non-null  object
 9   debt              21423 non-null  int64 
 10  total_income      21423 non-null  int64 
 11  purpose           21423 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

В столбцах 'days_employed' и 'total_income' изменили тип данных с вещественного на целочисленный универсальным методом astype()


<a id="duplicates"></a>
### Обработка дубликатов

In [6]:
#print(data['education'].unique()) # В столбще нашли одинаковые значения с разным написанием
data['education'] = data['education'].str.lower() # Приведем значения к единому виду  


#print(data.duplicated().sum()) # Дубликатами будем считать строки с одинаковыми занчениями во всех столбцах
data = data.drop_duplicates().reset_index(drop = True) # Удалим дубликаты
print(data.duplicated().sum()) # Дубликатов не осталось

0


**Вывод**

1. Привели одинаковые значения записанные в разном регистре к единому стилю
2. В датасете у нас нет имен, id или других идентификаторов клиентов, поэтому счидаем дублями строки с полным совпадением значений всех столбцов. Находим дубли применяя .duplicated() ко всему датафрейму. Вероятно данные поступили из нескольких учетных систем и часть данные некоторых клиентов хранились сразу в нескольких системах. Дубли удалили методом drop_duplicates() и перезаписали индексы. 

<a id="lemmatization"></a>
### Лемматизация

In [7]:
# В столбце 'purpose' одни и теже категории целей кредита описаны разными словами. Цель этого этапа обобщить категории


text = ','.join(data['purpose'].unique()) # Найдем уникальные значения столбца 'purpose', соберем в одну строку методом join() для лемматизации
lemmas = m.lemmatize(text) # Лемматизируем полученную строку

Counter(lemmas) # Посчитаем частоту упоминания каждого существительного, это и будут основные категории.
# Самые часто упоминаемые: недвижимость(жилье), автомобиль, образование, свадьба

# Напишем функцию для лемматизации значей столбца 'purpose' и определения категории
# В категорию 'недвижимость' мы должны определить все значение где упоминаются существительные 'недвижимость' или 'жилье'
# В категорию 'автомобиль' где употребляется'автомобиль' и в раздел'образование' - 'образование' соответственно.
# Для всех остальных вернем 'цель неопределена'
def lemmas_purpose(row):
    purpose = row['purpose']
    purpose_lemmas = m.lemmatize(purpose)
 
    if ('недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas):
        return 'недвижимость'
    elif 'автомобиль'in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    else:
        return 'цель не определена'
    
# Создадим новый столбец и сохнарим в него лемматизированные значения столбца 'purpose'
data['purpose_lemmas'] = data.apply(lemmas_purpose, axis=1)    
data['purpose_lemmas'].value_counts()


недвижимость    10763
автомобиль       4284
образование      3995
свадьба          2310
Name: purpose_lemmas, dtype: int64

**Вывод**

1. Определили категории кредитов. 
Для этого: 
- нашли и объединили в одну строку уникальные значения столбца 'purpose', 
- лемматизировали строку и определили самые популярные значения, которые и являются категориями кредитов
2. Написали функцию для лемматизации значей столбца 'purpose' и определения категории кредита для каждого клиента

<a id="categorization"></a>
### Категоризация данных

In [8]:
# Определим категории(уникальные значения) для каждого столбца  

#print(data['children'].value_counts()) # Категоризация по количеству детей

# Категоризация по стажу. Столбец содержит большое количесвто уникальных значений
# Следует разделить выделить категории стажа: до 1825 "маленький стаж", от 1825 до 7300 "средний стаж", от 7300 "большой стаж" 
# Напишем функцию для категоризации стажа
def seniority(days_employed):
    if days_employed <= 1825:
        return 'маленький стаж'
    if  days_employed > 1825 and days_employed < 7300:
        return 'средний стаж'
    return 'большой стаж'
data['seniority'] = data['days_employed'].apply(seniority)
#print(data['seniority'].value_counts()) # Категоризация по стажу.


# Категоризация по возрасту. Столбец также содержит большое количесвто уникальных значений
# Следует разделить выделить категории возраста до 30 лет - "молодежь" , от 30 до 60 "среднего возраста", больше 60 "пожилые"
def age(dob_years):
    if dob_years <= 30:
        return 'молодежь'
    if  dob_years > 30 and dob_years < 60:
        return 'среднего возраста'
    return 'пожилые'
data['age'] = data['dob_years'].apply(age)
#print(data['age'].value_counts()) # Категоризация по возрасту.

# Категоризация по образованию. 
#Для этого можно использовать значению одного из столбцов 'education' или 'education_id'
#print(data['education'].value_counts()) 
#print(data['education_id'].value_counts())
display(data[['education', 'education_id']].drop_duplicates().set_index('education_id')) # Или оба сразу

# Категоризация по семейному положению
# Может быть выполнена аналогично категоризации по образованию
#print(data['family_status'].value_counts())
#print(data['family_status_id'].value_counts())
display(data[['family_status', 'family_status_id']].drop_duplicates().set_index('family_status_id'))

#print(data['gender'].value_counts()) # Категоризация по половому признаку.

#print(data['income_type'].value_counts()) # Категоризация по типу занятости.

#print(data['debt'].value_counts()) # Категоризация по наличию/отсутствию задолженности в прошлом.

# Категоризация по уровню дохода. Столбец содержит большое количесвто уникальных значений.
# Мы не можем проводить исследования и делать выводы для каждого уникального значения, для этого мы должны классифицировать данные по схожим признакам. 
# Выделим категории по размеру дохода: до 75'000 "низкий доход", от 75'000 до 150'000 - "средний доход", свыше 150'000 - "высокий доход"
#print(data['total_income'].value_counts())
def income(total_income):
    if total_income <= 75000:
        return 'низкий доход'
    if  total_income > 75000 and total_income < 150000:
        return 'средний доход'
    return 'высокий доход'
data['income'] = data['total_income'].apply(income)
#print(data['income'].value_counts())

#print(data['purpose_lemmas'].value_counts()) # Категоризация по назначению кредита


,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


In [9]:
data['income'].value_counts()

высокий доход    10020
средний доход     9476
низкий доход      1856
Name: income, dtype: int64

**Вывод**

Для категоризации мы должны были определить уникальные значения, каждого столбца данных.
Для данных о стаже, возрасте и доходах мы ввели дополнительные параметры уровня (низкого, среднего и высокого) и произвели категоризацию по этим параметрам. 
<p>Во всех остальных случаях мы использовали методы .value_counts() или .drop_duplicates()


<a id="questions"></a>
## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
report_children = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
report_children['mean'] = (report_children['mean'] * 100).round(2) 
report_children.columns = ['debt', 'total', '%']
report_children = report_children.sort_values(by = '%', ascending = False)
report_children

,debt,total,%
children,,,
4,4,41,9.76
2,202,2114,9.56
1,442,4839,9.13
3,27,328,8.23
0,1058,14021,7.55
5,0,9,0.00


**Вывод**

Определенно наличие детей сказывается на возможность возврата кредита. 
<p>У бездетных самый низкий показатель задолженности (случаев с 5-ю детьми очень мало, нерепрезентативная выборка)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
report_family_status = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
report_family_status['mean'] = (report_family_status['mean'] * 100).round(2) 
report_family_status.columns = ['debt', 'total', '%']
report_family_status = report_family_status.sort_values(by = '%', ascending = False)
report_family_status

,debt,total,%
family_status,,,
Не женат / не замужем,273,2794,9.77
гражданский брак,386,4129,9.35
женат / замужем,927,12290,7.54
в разводе,85,1185,7.17
вдовец / вдова,62,954,6.50


**Вывод**

Зависимость семейного положения также прослеживается. К женатым доверия зачительно больше чем к неженатым. И даже у пар в гражданском браке показатели значительно хуже женатых.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
#report_dob = data.pivot_table(index = ['age'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
#report_dob['mean'] = (report_dob['mean'] * 100).round(2) 
#report_dob.columns = ['debt', 'total', '%']
#report_dob = report_dob.sort_values(by = '%', ascending = False)
#report_dob
report_income = data.pivot_table(index = ['income'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
report_income['mean'] = (report_income['mean'] * 100).round(2) 
report_income.columns = ['debt', 'total', '%']
report_income = report_income.sort_values(by = '%', ascending = False)
report_income

,debt,total,%
income,,,
средний доход,801,9476,8.45
высокий доход,796,10020,7.94
низкий доход,136,1856,7.33


**Вывод**

Результат неожиданый - лучшие показатели у клиентов с низким доходом, вероятно такие люди ответственнее в распределении доходов.
<p>Промежуточный резульат у клиентов с высоким доходом, что ожидаемо. 
<p>Больше всего просроченных кредитов у клиентов со средним доходом. Вероятно такой доход провацирует худший учет финансов. 

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
report_purpose = data.pivot_table(index = ['purpose_lemmas'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
report_purpose['mean'] = (report_purpose['mean'] * 100).round(2) 
report_purpose.columns = ['debt', 'total', '%']
report_purpose = report_purpose.sort_values(by = '%', ascending = False)
report_purpose

,debt,total,%
purpose_lemmas,,,
автомобиль,400,4284,9.34
образование,370,3995,9.26
свадьба,184,2310,7.97
недвижимость,779,10763,7.24


**Вывод**

Кредиты на свадьбу и недвижимость выплачивают стабильнее чем на автомобили и образование

<a id="conclusion"></a>
## Шаг 4. Общий вывод

В этом проекте мы должны были обработать входные данные от банка (статистика о платёжеспособности клиентов) и предоставить данные для построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
Для этого нам потребовалось ***подготовить данные для исследования***: 
1. Обработать пропуски в даннах
- Найдены нулевые значения в столбце 'dob_years' (возраст клиента). Значений было меньше одного процента, что значительно не повлияет на результаты. Принято решение удалить строки полностью.
- В столбце 'children'(количество детей) обрнаружили аномальные значения "20", где верояно пропущена точка (2.0) и "-1" с ошибочным знаком. Заменили.
- В столбще 'gender' обнаружено некорректное значение пола - "XNA". Встречается единожды, что не критично для исследования и принято решение удалить всю строку полностью.
- В столбце 'days_employed' браружили отрицательные и аномально большие значения.Аномально большие данные видимо указаны в часах и поступили из другой системы. В качестве верхней границы стажа приняли средний трудовой стаж в РФ при выходе на пенсию. Данные выше этой границы пересчитали в дни. В отрицательных значениях заменили знак.
- В столбцах 'days_employed' и 'total_income' обнаружили пропущенные значения. Значений достаточно много (окололо 10%), возможно проблема с выгрузкой данных и стоило бы пообщаться с разработчиками. Удалить такое количество строк без ущерба результату мы не можем. Принято решение заполнить пропуски медианным значением. Принято допущение, что трудовой стаж и ежемесячный доход напрямую зависят от возраста, поэтому пропущеные значения заполнены медианным значением с группировакой по переменной возраст.
2. Изменить некорректные типы данных
- В столбцах 'days_employed' и 'total_income' изменили тип данных с вещественного на целочисленный 
3. Обработать дубликаты
- Привели одинаковые значения записанные в разном регистре к единому стилю
- В датасете у нас нет имен, id или других идентификаторов клиентов, поэтому мы считли дублями строки с полным совпадением значений всех столбцов. Вероятно данные поступили из нескольких учетных систем и часть данные некоторых клиентов хранились сразу в нескольких системах. Дубли удалили.
4. Провести лемматизацию для обобщения категорий кредитов, т.к. в столбце 'purpose' одни и теже категории целей кредита описаны разными словами. Выделили четыре категории: недвижимость, автомобиль, образование, свадьба    
5. Провести категоризацию данных. Для исследования нас интересуют категории в данных о наличии детей, семейном положении, уровне дохода и возраст.Категоризация по уровню дохода потребовала провести дополнительную классификацию. Столбец содержит большое количесвто уникальных значений. Мы не можем проводить исследования и делать выводы для каждого уникального значения, для этого мы должны были классифицировать данные по схожим признакам. Выделили категории по размеру дохода: до 75 000 "низкий доход", от 75 000 до 150 000 - "средний доход", свыше 150 000 - "высокий доход"



***И ответить на вопросы:***
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

***Зависимости мы рассчитали в процентах, как отношение количества просроченных кредитов к общему числу выданных кредитов в каждой категории. Результаты исследования представили в таблицах***

Зависимость между ***наличием детей*** и возвратом кредита в срок

In [14]:
report_children

,debt,total,%
children,,,
4,4,41,9.76
2,202,2114,9.56
1,442,4839,9.13
3,27,328,8.23
0,1058,14021,7.55
5,0,9,0.00


Зависимость между ***семейным положением*** и возвратом кредита в срок

In [15]:
report_family_status

,debt,total,%
family_status,,,
Не женат / не замужем,273,2794,9.77
гражданский брак,386,4129,9.35
женат / замужем,927,12290,7.54
в разводе,85,1185,7.17
вдовец / вдова,62,954,6.50


Зависимость между ***уровнем дохода*** и возвратом кредита в срок

In [16]:
report_income

,debt,total,%
income,,,
средний доход,801,9476,8.45
высокий доход,796,10020,7.94
низкий доход,136,1856,7.33


Зависимость между ***целью кредита*** и возвратом его в срок

In [17]:
report_purpose

,debt,total,%
purpose_lemmas,,,
автомобиль,400,4284,9.34
образование,370,3995,9.26
свадьба,184,2310,7.97
недвижимость,779,10763,7.24
